In [7]:
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score, roc_curve
import sys
sys.path.append('/Users/ham/Desktop/research/LSHiForest/src/')
from detectors import LSHiForest

In [8]:
features_path = "NUSW-NB15_features.csv"
data_path = "UNSW-NB15_1.csv"

# === データ読み込み ===
df_features = pd.read_csv(features_path, encoding='cp1252')
feature_names = df_features['Name'].tolist()

df = pd.read_csv(data_path, header=None, low_memory=False)
df.columns = feature_names

# データ数制限（最初の10000件を使う）
df = df.iloc[:10000, :]

# 正常データと異常データを分離
normal_data = df[df['Label'] == 0]
anomaly_data = df[df['Label'] == 1]

# カテゴリカル・数値特徴量の指定
categorical_features = [
    'sport', 'dsport', 'proto', 'service', 'state', 'is_sm_ips_ports', 'is_ftp_login'
]
numerical_features = [
    "dur", "sbytes", "dbytes", "sttl", "dttl", "sloss", "dloss", "Sload", "Dload", "Spkts", "Dpkts",
    "swin", "dwin", "stcpb", "dtcpb", "smeansz", "dmeansz", "trans_depth", "res_bdy_len",
    "Sjit", "Djit", "Sintpkt", "Dintpkt", "tcprtt", "synack", "ackdat", "ct_state_ttl",
    "ct_flw_http_mthd", "ct_ftp_cmd", "ct_srv_src", "ct_srv_dst", "ct_dst_ltm",
    "ct_src_ltm", "ct_src_dport_ltm", "ct_dst_sport_ltm", "ct_dst_src_ltm"
]

# 正常データを学習・テストに分割
train_size = 0.8
normal_train = normal_data.sample(frac=train_size, random_state=42)
normal_test = normal_data.drop(normal_train.index)

# === 数値特徴量のビニング関数 ===
def bin_numerical_features(df, numerical_features, n_bins=10):
    df_binned = df.copy()
    for col in numerical_features:
        unique_vals = df[col].nunique()
        if unique_vals > n_bins:
            df_binned[col] = pd.cut(df[col], bins=n_bins, labels=False)
        else:
            df_binned[col] = df[col]
    return df_binned

# トレーニング・テストデータをビニング適用
train_binned = bin_numerical_features(normal_train, numerical_features)
test_binned = bin_numerical_features(pd.concat([normal_test, anomaly_data]), numerical_features)

# 特徴量リスト（カテゴリカル＋ビニング後の数値特徴量）
features_to_use = categorical_features + numerical_features

# 最終データセット作成
X_train = train_binned[features_to_use]
y_train = normal_train['Label']  # すべて正常

X_test = test_binned[features_to_use]
y_test = pd.concat([normal_test, anomaly_data])['Label']

# データサイズ出力
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Train normal count: {y_train.value_counts().to_dict()}")
print(f"Test label distribution: {y_test.value_counts().to_dict()}")


X_train shape: (7462, 43)
X_test shape: (2538, 43)
Train normal count: {0: 7462}
Test label distribution: {0: 1865, 1: 673}


In [9]:
num_ensemblers=200
classifiers = [("MinLSH", LSHiForest('MinLSH', num_ensemblers))]


for i, (clf_name, clf) in enumerate(classifiers):
	
	print("\n"+clf_name+":")
	start_time = time.time()
	
	clf.fit(X_train)
	
	train_time = time.time()-start_time
	
	y_pred = clf.decision_function(X_test)
	
	test_time = time.time()-start_time-train_time
	
	auc = roc_auc_score(y_test, y_pred)
	
	print("\tAUC score:\t", auc)
	print("\tTraining time:\t", train_time) 
	print("\tTesting time:\t", test_time)


MinLSH:


/Users/ham/Desktop/research/LSHiForest/src/detectors/lsh_tree.py:56: RuntimeWarning: invalid value encountered in scalar divide
  children[key] = self._recursive_build(child_data, depth_limit, min(0.0, (children_count[key]-mean)/std), cur_index+1)


	AUC score:	 0.22440554676949676
	Training time:	 24.048838138580322
	Testing time:	 72.98170399665833
